In [5]:
import warnings
warnings.filterwarnings('ignore')

import os
ROOT_DIR = os.getcwd()[:-28] # verify this points to the root directory of the project

from tools.modeling.deprecated.timeseriesmodeler import TimeSeriesModeler

from nba.common import add_game_log_lag, add_game_log_last_5
import nba.constants as c

from xgboost import XGBRegressor

import numpy as np
import pandas as pd


season = c.SEASON_2022_2023

ROOT_DIR

'/Users/rmcdevi2/Projects/personal/nba-analysis/'

In [6]:
logs = pd.read_csv(ROOT_DIR + '/data/game_logs/{season}/top_100_scorers_{season}_game_log.csv'.format(season=season))
logs

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,SEASON
0,2022-23,203954,Joel Embiid,1610612755,PHI,Philadelphia 76ers,22201198,2023-04-06,PHI vs. MIA,L,...,2,2,0,2,0,1,4,21,-11,2022-23
1,2022-23,203954,Joel Embiid,1610612755,PHI,Philadelphia 76ers,22201181,2023-04-04,PHI vs. BOS,W,...,6,3,0,2,1,3,9,52,17,2022-23
2,2022-23,203954,Joel Embiid,1610612755,PHI,Philadelphia 76ers,22201174,2023-04-02,PHI @ MIL,L,...,5,1,0,0,1,4,4,28,-10,2022-23
3,2022-23,203954,Joel Embiid,1610612755,PHI,Philadelphia 76ers,22201150,2023-03-31,PHI vs. TOR,W,...,1,5,0,1,0,3,6,25,0,2022-23
4,2022-23,203954,Joel Embiid,1610612755,PHI,Philadelphia 76ers,22201139,2023-03-29,PHI vs. DAL,W,...,3,4,1,1,1,4,6,25,10,2022-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6576,2022-23,1628386,Jarrett Allen,1610612739,CLE,Cleveland Cavaliers,22200072,2022-10-28,CLE @ BOS,W,...,2,0,0,0,0,2,2,12,25,2022-23
6577,2022-23,1628386,Jarrett Allen,1610612739,CLE,Cleveland Cavaliers,22200056,2022-10-26,CLE vs. ORL,W,...,1,1,1,1,0,5,5,18,11,2022-23
6578,2022-23,1628386,Jarrett Allen,1610612739,CLE,Cleveland Cavaliers,22200039,2022-10-23,CLE vs. WAS,W,...,3,0,0,2,1,2,4,15,-1,2022-23
6579,2022-23,1628386,Jarrett Allen,1610612739,CLE,Cleveland Cavaliers,22200032,2022-10-22,CLE @ CHI,W,...,2,2,0,1,0,2,3,8,5,2022-23


In [7]:
logs[c.GAME_DATE] = pd.to_datetime(logs[c.GAME_DATE])
logs = add_game_log_last_5(logs)
logs = add_game_log_lag(logs)

logs.sort_values(by=[c.GAME_DATE])
logs

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,FG_PCT_LAG,FGM_LAG,FGA_LAG,FG3M_LAG,FG3A_LAG,FG3_PCT_LAG,FTM_LAG,FTA_LAG,FT_PCT_LAG,PLUS_MINUS_LAG
552,2022-23,2544,LeBron James,1610612747,LAL,Los Angeles Lakers,22200002,2022-10-18,LAL @ GSW,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551,2022-23,2544,LeBron James,1610612747,LAL,Los Angeles Lakers,22200016,2022-10-20,LAL vs. LAC,L,...,0.462,12.0,26.0,3.0,10.0,0.300,4.0,4.0,1.000,-10.0
550,2022-23,2544,LeBron James,1610612747,LAL,Los Angeles Lakers,22200037,2022-10-23,LAL vs. POR,L,...,0.412,7.0,17.0,2.0,8.0,0.250,4.0,6.0,0.667,-1.0
549,2022-23,2544,LeBron James,1610612747,LAL,Los Angeles Lakers,22200064,2022-10-26,LAL @ DEN,L,...,0.545,12.0,22.0,2.0,9.0,0.222,5.0,7.0,0.714,2.0
548,2022-23,2544,LeBron James,1610612747,LAL,Los Angeles Lakers,22200076,2022-10-28,LAL @ MIN,L,...,0.381,8.0,21.0,2.0,8.0,0.250,1.0,2.0,0.500,-7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4800,2022-23,1631097,Bennedict Mathurin,1610612754,IND,Indiana Pacers,22201149,2023-03-31,IND vs. OKC,W,...,0.524,11.0,21.0,1.0,5.0,0.200,6.0,9.0,0.667,-11.0
4799,2022-23,1631097,Bennedict Mathurin,1610612754,IND,Indiana Pacers,22201173,2023-04-02,IND @ CLE,L,...,0.625,5.0,8.0,1.0,3.0,0.333,4.0,5.0,0.800,-9.0
4798,2022-23,1631097,Bennedict Mathurin,1610612754,IND,Indiana Pacers,22201190,2023-04-05,IND vs. NYK,L,...,0.417,5.0,12.0,1.0,2.0,0.500,8.0,8.0,1.000,-7.0
4797,2022-23,1631097,Bennedict Mathurin,1610612754,IND,Indiana Pacers,22201203,2023-04-07,IND vs. DET,L,...,0.353,6.0,17.0,1.0,3.0,0.333,4.0,6.0,0.667,-31.0


In [8]:
# Preprocess Data
PTS_PREDICTION_MASK = [c.PTS] + c.GAME_LOG_LAG_MASK + c.GAME_LOG_LAST_5_MASK
logs = logs[PTS_PREDICTION_MASK]

cats = logs.select_dtypes(exclude=np.number).columns.tolist()
for col in cats:
    logs[col] = logs[col].astype('category')

logs = logs.dropna()
logs

,PTS,MIN_LAG,PTS_LAG,REB_LAG,AST_LAG,STL_LAG,BLK_LAG,TOV_LAG,FG_PCT_LAG,FGM_LAG,...,FG_PCT_LAST_5,FGM_LAST_5,FGA_LAST_5,FG3M_LAST_5,FG3A_LAST_5,FG3_PCT_LAST_5,FTM_LAST_5,FTA_LAST_5,FT_PCT_LAST_5,PLUS_MINUS_LAST_5
547,26,37.440000,28.0,7.0,5.0,4.0,0.0,3.0,0.417,10.0,...,0.4434,9.8,22.0,2.0,8.2,0.2378,4.2,5.4,0.7512,-5.8
546,20,34.703333,26.0,6.0,8.0,1.0,0.0,3.0,0.455,10.0,...,0.4420,9.4,21.2,1.8,7.2,0.2578,4.2,5.6,0.7112,-4.0
545,17,39.716667,20.0,10.0,8.0,0.0,2.0,2.0,0.391,9.0,...,0.4378,9.8,22.4,1.4,7.0,0.2078,3.8,5.2,0.6778,-2.8
544,27,34.163333,17.0,10.0,8.0,0.0,1.0,3.0,0.368,7.0,...,0.4024,8.8,21.8,1.0,6.2,0.1634,3.4,4.8,0.6550,-5.4
543,30,33.816667,27.0,7.0,4.0,0.0,0.0,1.0,0.565,13.0,...,0.4392,9.8,22.2,0.8,5.4,0.1634,3.2,5.0,0.5550,-5.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4800,15,40.576667,29.0,9.0,2.0,2.0,0.0,0.0,0.524,11.0,...,0.4696,5.2,10.6,1.2,3.6,0.2900,3.6,4.4,0.5048,-7.6
4799,19,35.100000,15.0,4.0,0.0,0.0,0.0,1.0,0.625,5.0,...,0.4746,5.6,11.2,0.8,3.6,0.1566,3.2,4.2,0.4648,-10.6
4798,17,32.568333,19.0,9.0,0.0,0.0,2.0,4.0,0.417,5.0,...,0.5080,6.4,12.8,1.0,3.8,0.2566,4.8,5.8,0.6648,-7.0
4797,16,32.435000,17.0,8.0,4.0,0.0,0.0,6.0,0.353,6.0,...,0.4786,7.2,15.4,1.2,4.2,0.3232,5.6,7.0,0.7982,-11.2


In [9]:
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'n_estimators': 1000
}

In [10]:
ts_modeler = TimeSeriesModeler(XGBRegressor())
ts_modeler.boruta_feature_selection(params, logs, c.PTS)

Final Features confirmed as important: [['PTS_LAST_5', 'FGA_LAST_5', 'FTA_LAST_5'], ['PTS_LAST_5', 'FGA_LAST_5', 'FTA_LAST_5'], ['PTS_LAST_5', 'FGA_LAST_5', 'FTM_LAST_5', 'FTA_LAST_5'], ['PTS_LAST_5', 'FGA_LAST_5', 'FTM_LAST_5', 'FTA_LAST_5'], ['PTS_LAST_5', 'TOV_LAST_5', 'FGA_LAST_5', 'FTM_LAST_5', 'FTA_LAST_5']]
